In [20]:
#Enter Source Name
source_name=input('Enter Source name (Format="V1139_cyg"): ')

Enter Source name (Format="V1139_cyg"): v2494_cyg


In [32]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from sklearn.cluster import KMeans
from math import ceil

#Load and Analyze YSO Data

csv_path = fr"C:\Users\kgokh\Downloads\TAMOONS\MAIN CODES\selected_ysos_{source_name}.csv"
ysos_df = pd.read_csv(csv_path)

ra_center_deg = ysos_df['RA_deg'].median()
dec_center_deg = ysos_df['DEC_deg'].median()
coord_center = SkyCoord(ra=ra_center_deg*u.deg, dec=dec_center_deg*u.deg)
print(f" Using median center: RA={ra_center_deg}, DEC={dec_center_deg}")

#Convert to SkyCoord & Compute Offsets

ysos_df['coord'] = [SkyCoord(ra=ra*u.deg, dec=dec*u.deg)
                    for ra, dec in zip(ysos_df['RA_deg'], ysos_df['DEC_deg'])]
ysos_df['offset_arcmin'] = [c.separation(coord_center).arcminute for c in ysos_df['coord']]

#Filter offsets
ysos_df = ysos_df[(ysos_df['offset_arcmin'] > 0.3) & (ysos_df['offset_arcmin'] <= 6)]

#Bin by Jmag (rounded to nearest 0.5 mag)
ysos_df['Jmag_bin'] = ysos_df['Jmag'].apply(lambda x: round(x * 2) / 2)
magnitude_groups = ysos_df.groupby('Jmag_bin')

#Chunk into ≤8 per group

sublists = []
for mag, group in magnitude_groups:
    group = group.sort_values('offset_arcmin')
    chunks = [group.iloc[i:i+8].copy() for i in range(0, len(group), 8)]
    sublists.extend(chunks)

#Collision Check ≥ 0.3 arcmin

def collision_check(sublist, min_sep=1.0):
    valid_targets = []
    for _, row in sublist.iterrows():
        this_coord = row['coord']
        if all(this_coord.separation(other['coord']).arcminute >= min_sep for other in valid_targets):
            valid_targets.append(row)
    return pd.DataFrame(valid_targets)

collision_free_sublists = []
for sublist in sublists:
    filtered = collision_check(sublist)
    collision_free_sublists.append(filtered)

#Polar Clustering Conflict Separation (θ + r based)

def polar_clustering_separation(sublist, min_angle_sep=10.0, min_radial_sep=1.0):
    if len(sublist) <= 1:
        return [sublist]

    coord_center = SkyCoord(
        ra=sublist['RA_deg'].median() * u.deg,
        dec=sublist['DEC_deg'].median() * u.deg
    )

    sublist = sublist.copy()
    sublist['theta_deg'] = [
        (coord_center.position_angle(SkyCoord(ra=row['RA_deg']*u.deg, dec=row['DEC_deg']*u.deg)).deg) % 360
        for _, row in sublist.iterrows()
    ]
    sublist['radius_arcmin'] = [
        coord_center.separation(SkyCoord(ra=row['RA_deg']*u.deg, dec=row['DEC_deg']*u.deg)).arcminute
        for _, row in sublist.iterrows()
    ]

    separated_groups = []
    assigned = [False] * len(sublist)
    for i in range(len(sublist)):
        if assigned[i]:
            continue
        group = [sublist.iloc[i]]
        assigned[i] = True
        for j in range(i+1, len(sublist)):
            if assigned[j]:
                continue
            dtheta = abs(sublist.iloc[i]['theta_deg'] - sublist.iloc[j]['theta_deg'])
            if dtheta > 180:
                dtheta = 360 - dtheta
            drad = abs(sublist.iloc[i]['radius_arcmin'] - sublist.iloc[j]['radius_arcmin'])
            if dtheta >= min_angle_sep or drad >= min_radial_sep:
                group.append(sublist.iloc[j])
                assigned[j] = True
        separated_groups.append(pd.DataFrame(group))

    return separated_groups

# Apply Polar Clustering Separation
final_sublists = []
for group in collision_free_sublists:
    separated = polar_clustering_separation(group)
    final_sublists.extend(separated)

#If group still >8, apply KMeans

final_sublists_after_kmeans = []
for group in final_sublists:
    if len(group) > 8:
        coords_array = np.array([
            [c.ra.degree, c.dec.degree] for c in group['coord']
        ])
        k = ceil(len(group) / 8)
        kmeans = KMeans(n_clusters=k, random_state=42).fit(coords_array)
        group['cluster'] = kmeans.labels_
        for label in range(k):
            cluster_group = group[group['cluster'] == label].drop(columns='cluster')
            final_sublists_after_kmeans.append(cluster_group)
    else:
        final_sublists_after_kmeans.append(group)

#Output the Groups
output = []
for i, group in enumerate(final_sublists_after_kmeans, 1):
    for _, row in group.iterrows():
        coord = row['coord']
        ra_hms, dec_dms = coord.to_string('hmsdms').split()
        output.append({
            'Group': i,
            'GAIA_Source_ID': row['GAIA_Source_ID'],
            'RA_HMS': ra_hms,
            'DEC_DMS': dec_dms,
            'RA_deg': coord.ra.degree,
            'DEC_deg': coord.dec.degree,
            'Jmag': row['Jmag'],
            'Offset_arcmin': row['offset_arcmin']
        })

df_out = pd.DataFrame(output)
df_out.to_csv(f"grouped_ysos_{source_name}.csv", index=False)
print(f"✅ Done! Total groups: {len(final_sublists_after_kmeans)}. Results saved to 'grouped_ysos_{source_name}.csv'.")

#Visualization

preview_dir = f"group_previews_{source_name}"
os.makedirs(preview_dir, exist_ok=True)

colors = plt.cm.tab10(np.linspace(0, 1, len(final_sublists_after_kmeans)))
for i, group in enumerate(final_sublists_after_kmeans, 1):
    fig, ax = plt.subplots(figsize=(8, 8))
    group_coords = SkyCoord(ra=group['RA_deg'].values*u.deg, dec=group['DEC_deg'].values*u.deg)
    center_coord = SkyCoord(ra=ra_center_deg*u.deg, dec=dec_center_deg*u.deg)

    delta_ra = (group_coords.ra - center_coord.ra).arcmin
    delta_dec = (group_coords.dec - center_coord.dec).arcmin

    plt.scatter(delta_ra, delta_dec, label=f'Group {i}', color=colors[i % 10], s=50)
    plt.scatter(0, 0, c='black', marker='*', s=150, label='Center')

    x_ticks = np.linspace(-6, 6, 5)
    y_ticks = np.linspace(-6, 6, 5)
    plt.xticks(x_ticks, [f"{x:.1f}" for x in x_ticks])
    plt.yticks(y_ticks, [f"{y:.1f}" for y in y_ticks])

    plt.xlabel("ΔRA (arcmin)")
    plt.ylabel("ΔDEC (arcmin)")
    plt.title(f"Group {i} - 2MASS-K")
    plt.grid(True)
    plt.legend()
    plt.gca().invert_xaxis()
    plt.tight_layout()

    preview_path = os.path.join(preview_dir, f"group_{i}_preview_{source_name}.png")
    plt.savefig(preview_path, bbox_inches='tight')
    plt.close()

print(f"✅ Previews saved to folder: '{preview_dir}'")


 Using median center: RA=314.6416805, DEC=52.432675
✅ Done! Total groups: 16. Results saved to 'grouped_ysos_v2494_cyg.csv'.
✅ Previews saved to folder: 'group_previews_v2494_cyg'
